# 📊 Stock Performance Data Collection with yFinance

This notebook collects historical stock price data for selected S&P 500 companies using the **yfinance** API.
The goal is to build a clean, structured dataset that can later be used for:
- Financial analysis
- Visualization in Excel or Tableau
- Return and volatility calculations
- Event-based analysis

**Time period:** January 1, 2025 to July 1, 2025  (First Half of 2025)
**Sectors covered:** Technology and Banking

I used:
- **yfinance** to fetch stock market data  
- **pandas** for data manipulation  
- **datetime** to define the date range dynamically

In [1]:

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


Each ticker represents a publicly traded firm.
These companies were selected to compare **tech vs financial institutions**.

- Apple (AAPL)
- NVIDIA (NVDA)
- JPMorgan Chase (JPM)
- Bank of America (BAC)
- Goldman Sachs (GS)
- Morgan Stanley (MS)
- Citigroup (C)
- Wells Fargo (WFC)

In [2]:
tickers = ['AAPL', 'NVDA', 'JPM', 'BAC', 'GS', 'MS', 'C', 'WFC']

I then analyzed the **180 trading days** leading up to **July 1, 2025**.
This captures recent market behavior while avoiding short-term noise.

In [3]:
end_date = datetime(2025, 7, 1)
start_date = end_date - timedelta(days=180)


Used a list to temporarily store each stock's DataFrame.
Later, all datasets will be combined into a single master table.


In [4]:

all_data_list = []



For each ticker:
- Download historical prices
- Reset the index so Date becomes a column
- Ensure consistent column structure
- Add a Ticker column for identification

This step prevents downstream issues during visualization or analysis.


In [5]:

for ticker in tickers:
    try:
        data = yf.download(
            ticker,
            start=start_date,
            end=end_date,
            auto_adjust=True
        )

        data['Ticker'] = ticker
        data.reset_index(inplace=True)

        if 'Adj Close' not in data.columns:
            data['Adj Close'] = data['Close']

        required_columns = [
            'Date', 'Open', 'High', 'Low',
            'Close', 'Adj Close', 'Volume', 'Ticker'
        ]

        for col in required_columns:
            if col not in data.columns:
                if col == 'Adj Close':
                    data[col] = data['Close']
                else:
                    data[col] = 0

        data = data[required_columns]
        all_data_list.append(data)

        print(f"✅ Downloaded data for {ticker}")
    except Exception as e:
        print(f"❌ Error downloading {ticker}: {e}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


✅ Downloaded data for AAPL
✅ Downloaded data for NVDA


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


✅ Downloaded data for JPM
✅ Downloaded data for BAC


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


✅ Downloaded data for GS
✅ Downloaded data for MS


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

✅ Downloaded data for C
✅ Downloaded data for WFC




All individual stock tables are merged into a single DataFrame.
This format is ideal for:
- Tableau dashboards
- Excel pivot tables
- Python-based analysis


In [6]:

if all_data_list:
    all_data = pd.concat(all_data_list, ignore_index=True)

    print("Columns:", all_data.columns.tolist())
    print("Data shape:", all_data.shape)

    if isinstance(all_data.columns, pd.MultiIndex):
        all_data.columns = [
            col[0] if col[1] == '' else f"{col[0]}_{col[1]}"
            for col in all_data.columns
        ]

    all_data.to_excel('stock_data_raw.xlsx', index=False)

    print("✅ Data saved to stock_data_raw.xlsx")
    print("Total rows:", len(all_data))
else:
    print("❌ No data downloaded")


Columns: [('Date', ''), ('Open', 'AAPL'), ('High', 'AAPL'), ('Low', 'AAPL'), ('Close', 'AAPL'), ('Adj Close', ''), ('Volume', 'AAPL'), ('Ticker', ''), ('Open', 'NVDA'), ('High', 'NVDA'), ('Low', 'NVDA'), ('Close', 'NVDA'), ('Volume', 'NVDA'), ('Open', 'JPM'), ('High', 'JPM'), ('Low', 'JPM'), ('Close', 'JPM'), ('Volume', 'JPM'), ('Open', 'BAC'), ('High', 'BAC'), ('Low', 'BAC'), ('Close', 'BAC'), ('Volume', 'BAC'), ('Open', 'GS'), ('High', 'GS'), ('Low', 'GS'), ('Close', 'GS'), ('Volume', 'GS'), ('Open', 'MS'), ('High', 'MS'), ('Low', 'MS'), ('Close', 'MS'), ('Volume', 'MS'), ('Open', 'C'), ('High', 'C'), ('Low', 'C'), ('Close', 'C'), ('Volume', 'C'), ('Open', 'WFC'), ('High', 'WFC'), ('Low', 'WFC'), ('Close', 'WFC'), ('Volume', 'WFC')]
Data shape: (976, 43)
✅ Data saved to stock_data_raw.xlsx
Total rows: 976



## 📥 Importing Cleaned Excel Data into Python

After exporting the raw stock data to Excel, I used **Power Query Editor** to:
- Remove inconsistencies
- Standardize column names
- Validate missing values
- Prepare the dataset for analysis and visualization

This section imports the cleaned Excel file back into Python for validation
and additional calculations before visualization.

The Excel file is stored locally in the same project directory.
Using pandas, the cleaned dataset is loaded into a DataFrame.



In [8]:
df = pd.read_excel("stock_data.xlsx")
df.head()

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume Traded
0,2025-01-02,AAPL,248.330961,248.500565,241.238085,243.263199,243.263199,55740700
1,2025-01-02,BAC,43.606222,44.050781,43.270334,43.754410,43.754410,25610600
2,2025-01-02,C,69.893242,70.109997,68.622275,68.907997,68.907997,9827400
3,2025-01-02,GS,573.592419,578.543071,564.116865,569.295227,569.295227,2219000
4,2025-01-02,JPM,237.072695,239.395009,235.626169,236.167389,236.167389,9220900


After cleaning and structuring the raw stock price data in Excel, we proceed with exploratory analysis and visualization in Python using Matplotlib. This step helps us understand price movements, volatility, trading activity, and overall performance of each stock over the past six months. The cleaned dataset has been added to the GitHub repository to ensure reproducibility and version control.
